In [1]:
# =================================================================
# Celda 1: Configuración e Importaciones
# =================================================================

# Importación de librerías
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import config as cfg  # Importamos nuestro archivo de configuración

# Opcional: Recarga automática de módulos
%load_ext autoreload
%autoreload 2

In [11]:
# =================================================================
# Celda 2: Definición de Activos y Período (Modificada)
# =================================================================

# Lista de tickers de las empresas
company_tickers = [cfg.EMPRESA_PRINCIPAL, 'AAPL', 'IBM', 'ORCL', 'CSCO']

# Lista de tickers de los índices de referencia
index_tickers = ['^GSPC', '^NDX', 'XLK'] 

# Combinamos ambas listas para la descarga
all_tickers = company_tickers + index_tickers

# Definición del período de análisis (últimos 5 años)
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

# --- Verificación ---
print("Empresas a analizar:")
print(company_tickers)
print("\nÍndices de referencia:")
print(index_tickers)
print(f"\nPeríodo de análisis: Desde {start_date_str} hasta {end_date_str}")

Empresas a analizar:
['MSFT', 'AAPL', 'IBM', 'ORCL', 'CSCO']

Índices de referencia:
['^GSPC', '^NDX', 'XLK']

Período de análisis: Desde 2020-08-12 hasta 2025-08-11


In [12]:
# =================================================================
# Celda 3: Descarga de Datos Históricos (Modificada)
# =================================================================

# Descargamos los datos históricos para todos los tickers
try:
    # Descargamos el historial completo
    data_completa = yf.download(all_tickers, start=start_date_str, end=end_date_str)
    
    # Seleccionamos solo la columna 'Close', que ya está ajustada por defecto
    data = data_completa['Close']
    
    print("Descarga de datos completada exitosamente.")

except Exception as e:
    print(f"Ocurrió un error al descargar los datos: {e}")

# Revisamos los primeros 5 datos para verificar la descarga
print("\nPrimeras 5 filas del dataset combinado:")
display(data.head())

C:\Users\valen\AppData\Local\Temp\ipykernel_32640\3381844853.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_completa = yf.download(all_tickers, start=start_date_str, end=end_date_str)
[*********************100%***********************]  8 of 8 completed

Descarga de datos completada exitosamente.

Primeras 5 filas del dataset combinado:


Ticker,AAPL,CSCO,IBM,MSFT,ORCL,XLK,^GSPC,^NDX
Date,,,,,,,,
2020-08-12,110.051590,41.372978,97.608612,200.421906,50.410393,109.358109,3380.350098,11157.719727
2020-08-13,111.999245,36.745399,96.322067,199.952454,50.270802,109.406090,3373.429932,11178.370117
2020-08-14,111.899429,36.556179,96.506943,200.144104,50.438309,109.329292,3372.850098,11164.450195
2020-08-17,111.607262,36.203506,95.867531,201.466278,50.242886,110.058929,3381.989990,11288.570312
2020-08-18,112.537277,36.108891,96.237297,202.625534,51.350296,110.471741,3389.780029,11399.030273


In [13]:
# =================================================================
# Celda 4: Limpieza y Guardado por Separado (Modificada)
# =================================================================

# --- Metodología de Limpieza ---
print("Metodología de Limpieza de Datos:")
print("1. Se verifica la presencia de valores nulos (NaN) en el dataset.")
print("2. Se aplica la estrategia 'forward fill' (ffill) para rellenar los NaNs.")

# Verificamos valores nulos ANTES de la limpieza
print("\nConteo de valores nulos ANTES de la limpieza:")
print(data.isnull().sum())

# Aplicamos el forward fill al dataset completo
data.fillna(method='ffill', inplace=True)
# Adicionalmente, rellenamos hacia atrás por si hay NaNs al inicio
data.fillna(method='bfill', inplace=True)

print("\nConteo de valores nulos DESPUÉS de la limpieza:")
print(data.isnull().sum())

# --- Separación y Guardado de Datos ---

# Creamos un DataFrame solo con los datos de las empresas
company_data = data[company_tickers]

# Creamos un DataFrame solo con los datos de los índices
index_data = data[index_tickers]

# Guardamos los datasets en archivos CSV separados
company_output_path = 'datos_empresas.csv'
index_output_path = 'datos_indices.csv'

company_data.to_csv(company_output_path)
index_data.to_csv(index_output_path)

print(f"\n✅ Dataset de empresas guardado en: '{company_output_path}'")
display(company_data.head())
print(f"\n✅ Dataset de índices guardado en: '{index_output_path}'")
display(index_data.head())

Metodología de Limpieza de Datos:
1. Se verifica la presencia de valores nulos (NaN) en el dataset.
2. Se aplica la estrategia 'forward fill' (ffill) para rellenar los NaNs.

Conteo de valores nulos ANTES de la limpieza:
Ticker
AAPL     0
CSCO     0
IBM      0
MSFT     0
ORCL     0
XLK      0
^GSPC    0
^NDX     0
dtype: int64

Conteo de valores nulos DESPUÉS de la limpieza:
Ticker
AAPL     0
CSCO     0
IBM      0
MSFT     0
ORCL     0
XLK      0
^GSPC    0
^NDX     0
dtype: int64

✅ Dataset de empresas guardado en: 'datos_empresas.csv'


C:\Users\valen\AppData\Local\Temp\ipykernel_32640\316440047.py:15: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\valen\AppData\Local\Temp\ipykernel_32640\316440047.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.fillna(method='ffill', inplace=True)
C:\Users\valen\AppData\Local\Temp\ipykernel_32640\316440047.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill', inplace=True)
C:\Users\valen\AppData\Local\Temp\ipykernel_32640\316440047.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

Ticker,MSFT,AAPL,IBM,ORCL,CSCO
Date,,,,,
2020-08-12,200.421906,110.051590,97.608612,50.410393,41.372978
2020-08-13,199.952454,111.999245,96.322067,50.270802,36.745399
2020-08-14,200.144104,111.899429,96.506943,50.438309,36.556179
2020-08-17,201.466278,111.607262,95.867531,50.242886,36.203506
2020-08-18,202.625534,112.537277,96.237297,51.350296,36.108891



✅ Dataset de índices guardado en: 'datos_indices.csv'


Ticker,^GSPC,^NDX,XLK
Date,,,
2020-08-12,3380.350098,11157.719727,109.358109
2020-08-13,3373.429932,11178.370117,109.406090
2020-08-14,3372.850098,11164.450195,109.329292
2020-08-17,3381.989990,11288.570312,110.058929
2020-08-18,3389.780029,11399.030273,110.471741
